# 5. PyLapi Generator Automation

In our [last tutorial](./4.%20PyLapi%20Advanced%20with%20Asana.ipynb), we created the root `aAPI` class with five methods: `list()`, `load()`, `update()`, `create()`, and `delete()`. We also created two resource classes: `ProjectResource` and `TaskResource`.

In this tutorial, we generalise this design to all resource classes by putting this piece of custom code into [./aapi_rewrite.py](./aapi_rewrite.py) and letting the PyLapi generator merge the custom code into the generated `aAPI` class automatically.

---
Let us illustrate the PyLapi generator configuration and the code-rewrite feature, which we have briefly discussed in the [first tutorial](./1.%20Getting%20Started%20with%20PyLapi.ipynb).

Here is the `aAPI` configuration file [./aapi_config.py](./aapi_config.py):

In [1]:
!cat ./aapi_config.py

# aAPI (Asana API) PyLapi Generator Configuration

api_class_name = "aAPI"
output_py_name = "./aapi.py"
oas_file_name = "./asana_oas.json"

guide_attrs = {
    "summary",
    "description",
    "parameters",
    "request_body",
    # "ref",
}

naming = {
    "class_name": "upperCamel(singular($.path_segments[0])) + 'Resource'",
    "resource_name": "snake(singular($.path_segments[0]))",
    "class_path": "$.path_segments[0]",
    "resource_path": "re.sub(r'{[a-zA-Z0-9_-]+gid}', '{gid}', '/'.join($.path_segments[1:]))",
    "method_name": "lowerCamel($.operation_id)",
}

resource_class_args = {"gid": "$.gid"}

resource_method_args = {"send": {"data": "$"}, "give": "$.data"}

code_rewrite_file_name = "./aapi_rewrite.py"


In the configuration file [./aapi_config.py](./aapi_config.py) shown above, we are generating an API class called `aAPI` and storing the Python script in [./aapi.py](./aapi.py).
The OAS (OpenAPI Specification) is read from [./asana_oas.json](./asana_oas.json).

There are a few more settings:

- `guide_attrs`: The OAS attributes included into the script as inline comments, which serve as a user guide.
  - You may uncomment "ref" to dereference the OAS attributes if you don't mind a lengthy script file.
<br/><br/>

- `naming`: The conversion of class names, resource names, class paths, resource paths, and method names from OAS specifications
  - We use the singular class name and resource name (e.g., "project" instead of projects) here to make the code more readable.
    - This is valid as long as the API does not use both singular and plural resource names in the lead segment of the API path, like "/project" and "/projects".
    - The PyLapi generator will check and produce an error if class paths overlap.
  - The setting `"resource_path": "re.sub(...)` replaces any resource-specific global ID (gid) with a general one, e.g., `{project_gid}` into `{gid}`, for resource attribute mapping (to be discussed later).
<br/><br/>

- `resourdce_class_args`: Extra arguments for the `@aAPI.resource_class` decorator
  - The Asana API uses `gid` (global ID) to identify resources. The setting `resource_class_args = {"gid": "$.gid"}` maps the `.data.gid` attribute of the object to the `.gid` attribute, so it can be used as implicit arguments.
<br/><br/>

- `resource_method_args`: Extra arguments for the `@aAPI.resource_method` decorator
  - The Asana API puts the resource data into the "data" attribute in the request body and the response text. The `send` and `give` decorator arguments will apply generally to all methods, as in `resource_method_args = {"send": {"data": "$"}, "give": "$.data"}`.
<br/><br/>

- `code_rewrite_file_name`: Where the rewritten code is stored
  - The `aAPI` class we manually created in the [last tutorial](./4.%20PyLapi%20Advanced%20with%20Asana.ipynb) is saved in [./aapi_rewrite.py](./aapi_rewrite.py).

In [2]:
!cat ./aapi_rewrite.py

class aAPI(PyLapi):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.api_url = "https://app.asana.com/api/1.0"

    @PyLapi.resource_method(give="$.data")
    def list(self): pass

    @PyLapi.resource_method(method_path="{gid}", http_method="GET", give="$.data", load="$.data")
    def load(self): pass

    @PyLapi.resource_method(method_path="{gid}", http_method="PUT", send={"data": "$"}, give="$.data", load="$.data")
    def update(self): pass

    @PyLapi.resource_method(http_method="POST", send={"data": "$"}, give="$.data", load="$.data")
    def create(self): pass

    @PyLapi.resource_method(method_path="{gid}", http_method="DELETE", give=None)
    def delete(self): pass


---
Now we run the PyLapi generator to generate the `aAPI` Python script, with the rewritten code above merged into the script automatically.

In [3]:
!../tools/pylapi-autogen aapi_config.py

aAPI generated, merged with ./aapi_rewrite.py, and saved in ./aapi.py


---
The `aAPI` example here is a realistic use case of `PyLapi`: You

- automatically generate a Python API using the PyLapi generator,
- customise the code to your needs in a rewrite file,
- update the configuration with `code_rewrite_file_name`, and
- rerun the PyLapi generator to merge the custom code into the Python API.

With `aAPI` ready to use, we are now well set to explore it and see how we can use it to manage [resource data](./6.%20Resource%20Data.ipynb) and [search and modify](./8.%20Search%20and%20Modify.ipynb) them, as you would in real life.

## End of page